In [74]:
#Basic working model using collaborative filtering with watched, this is just removing the watched movies from list and rating genres from watched history

In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import ast

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:

# Sample user data
user_data = {
    'id': [1, 2, 3],
    'happy_movie': [['Drama', 'Comedy'], ['Romance', 'Family'], ['Adventure']],
    'sad_movie': [['Drama', 'Thriller'], ['Crime'], ['Horror', 'Science Fiction']],
    'neutral_movie': [['Comedy'], ['Action', 'Adventure'], ['Crime']],
    'fav_movie': ['The Godfather', "Schindler's List", 'The Shawshank Redemption']
}
user = pd.DataFrame(user_data)


# Load movie data from the specified file
file_path = '/content/drive/MyDrive/final project sample data/tmdb2024.csv'
movie1 = pd.read_csv(file_path, index_col=0)
#movie = minfo[['Movie_id', 'title', 'Genres', 'popularity', 'vote_average', 'vote_count']]



# Sample watched data
watched_data = {
    'userid': [1, 2, 3, 1, 2],
    'movieid': [238, 278, 240, 19404, 122],
    'rating_given': [8.0, 5.5, 9.5, 7.0, 6.5],
    'watching_mood': ['Happy', 'Sad', 'Neutral', 'Happy', 'Neutral']
}
watched = pd.DataFrame(watched_data)

In [78]:
movie1

,id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
0,27205,Inception,8.364,34495,tt1375666,83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"['Action', 'Science Fiction', 'Adventure']",2010,mid
1,157336,Interstellar,8.417,32571,tt0816692,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"['Adventure', 'Drama', 'Science Fiction']",2014,mid
2,155,The Dark Knight,8.512,30619,tt0468569,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,"['Drama', 'Action', 'Crime', 'Thriller']",2008,old
3,19995,Avatar,7.573,29815,tt0499549,79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",2009,old
4,24428,The Avengers,7.710,29166,tt0848228,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,"['Science Fiction', 'Action', 'Adventure']",2012,mid
...,...,...,...,...,...,...,...,...,...,...
1019194,673471,Atanker Choya,0.000,0,tt10452806,0.600,/zMVHYoIzXe25rtImORhU5po0rk1.jpg,"['Adventure', 'Fantasy', 'Horror']",2019,mid
1019200,673450,Pappa Lap 'n Verhaal van n pa en sy dogter,0.000,0,tt2027187,0.600,/oz0l9ZkVBMdFb9gTaE1TGk6T2of.jpg,['Drama'],1971,old
1019201,673453,Doctor Who: The Pertwee Years,0.000,0,tt0297086,0.600,/lrSEDtfHMzFsRCXEuOjw8oiwLVh.jpg,['Documentary'],1992,old
1019202,673455,The Taiga Troopers,0.000,0,tt0471935,0.627,/3jxhKTtZUwagmtEiv6tGaR5ZIuF.jpg,['Drama'],1966,old


In [79]:
pref_era='latest'

In [80]:
movie = movie1[movie1['Release_Era'] == pref_era]


In [81]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24891 entries, 282 to 1019212
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            24891 non-null  int64  
 1   title         24891 non-null  object 
 2   vote_average  24891 non-null  float64
 3   vote_count    24891 non-null  int64  
 4   imdb_id       24834 non-null  object 
 5   popularity    24891 non-null  float64
 6   poster_path   24861 non-null  object 
 7   Genres        24891 non-null  object 
 8   Release_Year  24891 non-null  int64  
 9   Release_Era   24891 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 2.1+ MB


In [82]:
movie

,id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
282,76600,Avatar: The Way of Water,7.654,9830,tt1630029,241.285,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,"['Science Fiction', 'Adventure', 'Action']",2022,latest
366,414906,The Batman,7.705,8584,tt1877830,100.817,/74xTEgt7R36Fpooo50r9T25onhq.jpg,"['Crime', 'Mystery', 'Thriller']",2022,latest
403,453395,Doctor Strange in the Multiverse of Madness,7.355,7991,tt9419884,96.530,/9Gtg2DzBhmYamXBS1hKAhiwbBKS.jpg,"['Fantasy', 'Action', 'Adventure']",2022,latest
444,361743,Top Gun: Maverick,8.260,7546,tt1745960,126.291,/62HCnUTziyWcpDaBO2i1DX17ljH.jpg,"['Action', 'Drama']",2022,latest
527,502356,The Super Mario Bros. Movie,7.775,6743,tt6718170,410.411,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,"['Animation', 'Family', 'Adventure', 'Fantasy'...",2023,latest
...,...,...,...,...,...,...,...,...,...,...
1018850,674254,Yatagan,0.000,0,tt5788218,0.626,/jv3DxblgXKFT40ZrifSALEw1bc.jpg,['Comedy'],2022,latest
1019058,673591,Gina,0.000,0,tt1365464,0.905,/due4NrXXD6MS6B4NdN4pDULGtgr.jpg,['Thriller'],2023,latest
1019060,673593,Mean Girls Musical,0.000,0,tt11762114,6.046,/1Lac9qrN6N4uO26T7nBgi8A7hVh.jpg,['Comedy'],2024,latest
1019124,673629,Black Wolf,0.000,0,tt10393330,0.616,/1E2wJPuaC8lN9EerSrN4Hwi2oa8.jpg,"['Action', 'Crime', 'Drama']",2022,latest


In [83]:
# Convert the string representation of list to actual list then to list of dictionaries
movie['Genres'] = movie['Genres'].apply(lambda x: [{'genre': genre} for genre in ast.literal_eval(x)])

# Extract the names of the genres from the list of dictionaries
movie['Genres'] = movie['Genres'].apply(lambda x: [i['genre'] for i in x])

<ipython-input-83-7ee3f36e7acf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['Genres'] = movie['Genres'].apply(lambda x: [{'genre': genre} for genre in ast.literal_eval(x)])
<ipython-input-83-7ee3f36e7acf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['Genres'] = movie['Genres'].apply(lambda x: [i['genre'] for i in x])


In [84]:
movie.head(2)

,id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
282,76600,Avatar: The Way of Water,7.654,9830,tt1630029,241.285,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,"[Science Fiction, Adventure, Action]",2022,latest
366,414906,The Batman,7.705,8584,tt1877830,100.817,/74xTEgt7R36Fpooo50r9T25onhq.jpg,"[Crime, Mystery, Thriller]",2022,latest


In [85]:
movie_genres = movie.copy()

In [86]:
movie_genres.rename(columns={'id': 'Movie_id'}, inplace=True)

In [87]:
movie_genres

,Movie_id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
282,76600,Avatar: The Way of Water,7.654,9830,tt1630029,241.285,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,"[Science Fiction, Adventure, Action]",2022,latest
366,414906,The Batman,7.705,8584,tt1877830,100.817,/74xTEgt7R36Fpooo50r9T25onhq.jpg,"[Crime, Mystery, Thriller]",2022,latest
403,453395,Doctor Strange in the Multiverse of Madness,7.355,7991,tt9419884,96.530,/9Gtg2DzBhmYamXBS1hKAhiwbBKS.jpg,"[Fantasy, Action, Adventure]",2022,latest
444,361743,Top Gun: Maverick,8.260,7546,tt1745960,126.291,/62HCnUTziyWcpDaBO2i1DX17ljH.jpg,"[Action, Drama]",2022,latest
527,502356,The Super Mario Bros. Movie,7.775,6743,tt6718170,410.411,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,"[Animation, Family, Adventure, Fantasy, Comedy]",2023,latest
...,...,...,...,...,...,...,...,...,...,...
1018850,674254,Yatagan,0.000,0,tt5788218,0.626,/jv3DxblgXKFT40ZrifSALEw1bc.jpg,[Comedy],2022,latest
1019058,673591,Gina,0.000,0,tt1365464,0.905,/due4NrXXD6MS6B4NdN4pDULGtgr.jpg,[Thriller],2023,latest
1019060,673593,Mean Girls Musical,0.000,0,tt11762114,6.046,/1Lac9qrN6N4uO26T7nBgi8A7hVh.jpg,[Comedy],2024,latest
1019124,673629,Black Wolf,0.000,0,tt10393330,0.616,/1E2wJPuaC8lN9EerSrN4Hwi2oa8.jpg,"[Action, Crime, Drama]",2022,latest


In [88]:
# prompt: Using dataframe movie: i want Genres to be reformatted with only list of name of genres, right now it has id and name and addtional text

# import ast

# # Create a copy of the dataframe
# movie_genres = movie.copy()

# # Convert the Genres column from string to list of dictionaries
# movie_genres['Genres'] = movie_genres['Genres'].apply(ast.literal_eval)

# # Extract the names of the genres from the list of dictionaries
# movie_genres['Genres'] = movie_genres['Genres'].apply(lambda x: [i['name'] for i in x])

# # Display the first 10 rows of the dataframe
# movie_genres.head(10)


In [89]:
# Assuming user, watched, and movie DataFrames are already defined as per your description.

# Merge the watched movies data with the movie details
watched_movies = pd.merge(watched, movie_genres, left_on='movieid', right_on='Movie_id')



In [90]:
watched_movies

,userid,movieid,rating_given,watching_mood,Movie_id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era


In [91]:

from itertools import chain

# Assuming 'user' is your DataFrame containing user preferences.
# Sample data structure for 'user' DataFrame is provided in your initial description.

# Function to expand and tag mood to genres
def expand_user_genres(user_df, mood_column):
    # Explode the DataFrame by the specified mood to handle multiple genres
    expanded_df = user_df[['id', mood_column]].explode(mood_column)
    # Assign the current mood as a new column
    expanded_df['mood'] = mood_column.split('_')[0].capitalize()
    # Rename the genre column to a generic 'genre' column
    expanded_df = expanded_df.rename(columns={mood_column: 'genre'})
    return expanded_df

# Process each mood
happy_genres = expand_user_genres(user, 'happy_movie')
sad_genres = expand_user_genres(user, 'sad_movie')
neutral_genres = expand_user_genres(user, 'neutral_movie')

# Concatenate all moods together
all_genres = pd.concat([happy_genres, sad_genres, neutral_genres])

# Optional: Reset index if you want a clean DataFrame index
all_genres.reset_index(drop=True, inplace=True)

all_genres.head()


,id,genre,mood
0,1,Drama,Happy
1,1,Comedy,Happy
2,2,Romance,Happy
3,2,Family,Happy
4,3,Adventure,Happy


In [92]:
from sklearn.preprocessing import MultiLabelBinarizer

# Assuming movie_genres DataFrame has a 'Genres' column with lists of genres
mlb = MultiLabelBinarizer()
movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
movie_genres_df = pd.DataFrame(movie_genres_encoded, columns=mlb.classes_, index=movie_genres['Movie_id'])



In [93]:
movie_genres_df

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Movie_id,,,,,,,,,,,,,,,,,,,
76600,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
414906,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
453395,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
361743,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
502356,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674254,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
673591,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
673593,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
# def recommend_movies_with_ratings_and_mood(user_id, current_mood, top_n=5):
#     # Get genres the user prefers in the current mood
#     mood_genres = all_genres[(all_genres['id'] == user_id) & (all_genres['mood'] == current_mood)]
#     user_pref_genres = mood_genres['genre'].unique()

#     # Get movies watched and rated by the user in the specified mood
#     mood_watched_movies = watched_movies[(watched_movies['userid'] == user_id) & (watched_movies['watching_mood'].str.capitalize() == current_mood)]

#     # Calculate average ratings for genres in this mood

#     mood_watched_movies_details = mood_watched_movies.merge(movie_genres, left_on='movieid', right_index=True)

#     # Adjusted section to use 'Genres_x'
#     genre_ratings = {}
#     for genre in user_pref_genres:
#         # Here we filter using 'Genres_x'
#         genre_movies = mood_watched_movies_details[mood_watched_movies_details['Genres_x'].apply(lambda x: genre in x)]
#         avg_rating = genre_movies['rating_given'].mean()
#         genre_ratings[genre] = avg_rating if not np.isnan(avg_rating) else 0


#     # Rank genres by average rating
#     ranked_genres = sorted(genre_ratings, key=genre_ratings.get, reverse=True)

#     # Encode user preferences weighted by average genre rating
#     user_pref_encoded = mlb.transform([ranked_genres])

#     # Calculate similarity between weighted user preferences and each movie's genres
#     similarity = cosine_similarity(user_pref_encoded, movie_genres_encoded)

#     # Exclude movies already watched
#     watched_movie_ids = watched_movies[watched_movies['userid'] == user_id]['movieid'].unique()
#     potential_movies = movie_genres[~movie_genres.index.isin(watched_movie_ids)]

#     # Get top N recommended movie IDs based on similarity scores, excluding watched movies
#     recommended_movie_ids = similarity.argsort()[0][-top_n:][::-1]

#     # Ensure correct indexing and data type consistency for recommended_movie_ids
#     recommended_movie_ids = [potential_movies.index[i] for i in recommended_movie_ids if i < len(potential_movies)]

#     # Map movie IDs back to movie titles and genres
#     recommended_movies_info = movie_genres.loc[recommended_movie_ids][['title', 'Genres']]

#     # Create a more readable output format
#     recommended_movies_output = recommended_movies_info.apply(lambda x: f"{x['title']} (Genres: {', '.join(x['Genres'])})", axis=1)

#     return recommended_movies_output.tolist()

# # Example usage
# recommended_movies = recommend_movies_with_ratings_and_mood(1, 'Happy', 5)
# for movie in recommended_movies:
#     print(movie)


In [95]:
def recommend_movies_with_weighted_genres(user_id, current_mood, top_n=5):
    mood_genres = all_genres[(all_genres['id'] == user_id) & (all_genres['mood'] == current_mood)]
    user_pref_genres = mood_genres['genre'].unique()

    # Check for mood-specific watched movies
    mood_watched_movies = watched_movies[(watched_movies['userid'] == user_id) & (watched_movies['watching_mood'].str.capitalize() == current_mood)]

    genre_ratings = {}
    if not mood_watched_movies.empty:
        # Calculate average ratings for genres from mood-specific watched movies
        mood_watched_movies_details = mood_watched_movies.merge(movie_genres, left_on='movieid', right_index=True)
        for genre in user_pref_genres:
            genre_movies = mood_watched_movies_details[mood_watched_movies_details['Genres_x'].apply(lambda x: genre in x)]
            avg_rating = genre_movies['rating_given'].mean()
            genre_ratings[genre] = avg_rating if not np.isnan(avg_rating) else 0
        # Normalize ratings to use as weights
        max_rating = max(genre_ratings.values(), default=1)
        genre_weights = {genre: rating / max_rating for genre, rating in genre_ratings.items()}
    else:
        genre_weights = {genre: 1 for genre in user_pref_genres}  # Default weight if no ratings

    # Encode movie genres and user preferences
    mlb = MultiLabelBinarizer()
    movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
    genre_prefs_encoded = mlb.transform([user_pref_genres])

    # Calculate similarity with weighted genres
    similarity_scores = cosine_similarity(genre_prefs_encoded, movie_genres_encoded)[0]
    weighted_similarity_scores = np.zeros_like(similarity_scores)

    for i, genre in enumerate(mlb.classes_):
        if genre in genre_weights:
            weighted_similarity_scores += similarity_scores * movie_genres_encoded[:, i] * genre_weights[genre]

    # Exclude watched movies
    watched_movie_ids = watched_movies[watched_movies['userid'] == user_id]['movieid'].unique()
    unwatched_indices = [i for i, movie_id in enumerate(movie_genres.index) if movie_id not in watched_movie_ids]

    # Recommend based on weighted similarity scores
    unwatched_similarity_scores = weighted_similarity_scores[unwatched_indices]
    top_indices = np.argsort(unwatched_similarity_scores)[-top_n:][::-1]

    recommended_movie_ids = [movie_genres.index[unwatched_indices[i]] for i in top_indices]
    recommended_movies_info = movie_genres.loc[recommended_movie_ids]
    recommended_movies_output = recommended_movies_info.apply(lambda x: f"{x['title']} (Genres: {', '.join(x['Genres'])})", axis=1).tolist()

    return recommended_movies_output

# Example usage
recommended_movies = recommend_movies_with_weighted_genres(1, 'Happy', 10)
for movie in recommended_movies:
    print(movie)



The Good Half (Genres: Comedy, Drama)
Dance Still (Genres: Drama, Comedy)
Le Principal (Genres: Drama, Comedy)
Jerk Off Session (Genres: Comedy, Drama)
Tarla (Genres: Comedy, Drama)
Ham (Genres: Drama, Comedy)
Réveillon (Genres: Comedy, Drama)
Liquid Bread (Genres: Drama, Comedy)
Member Rameshan 9aam Ward (Genres: Comedy, Drama)
Year of the Rat (Genres: Drama, Comedy)


In [96]:
# Example: Recommend movies for user 1 in a "Happy" mood, including genres
recommended_movies = recommend_movies_with_weighted_genres(1, 'Happy', 10)
for movie in recommended_movies:
    print(movie)

The Good Half (Genres: Comedy, Drama)
Dance Still (Genres: Drama, Comedy)
Le Principal (Genres: Drama, Comedy)
Jerk Off Session (Genres: Comedy, Drama)
Tarla (Genres: Comedy, Drama)
Ham (Genres: Drama, Comedy)
Réveillon (Genres: Comedy, Drama)
Liquid Bread (Genres: Drama, Comedy)
Member Rameshan 9aam Ward (Genres: Comedy, Drama)
Year of the Rat (Genres: Drama, Comedy)


In [97]:
recommended_movies = recommend_movies_with_weighted_genres(1, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

Bury Your Fish (Genres: Thriller, Drama)
The Date (Genres: Thriller, Drama)
Blocked (Genres: Drama, Thriller)
Clyde (Genres: Thriller, Drama)
Naalam Mura (Genres: Drama, Thriller)
Níobe (Genres: Thriller, Drama)
Coisa Pública (Genres: Drama, Thriller)
Silencioso Desespero (Genres: Drama, Thriller)
Disappear Into the Blue (Genres: Drama, Thriller)
Global Harmony (Genres: Drama, Thriller)


In [98]:
recommended_movies = recommend_movies_with_weighted_genres(1, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

Within a Week (Genres: Comedy)
The Gentlemen's Club (Genres: Comedy)
The Hater (Genres: Comedy)
Panik (Genres: Comedy)
Ragazzaccio (Genres: Comedy)
De Tatta's (Genres: Comedy)
Phantom Project (Genres: Comedy)
The Redeemables (Genres: Comedy)
Love in Varapuzha (Genres: Comedy)
A Fada do Lar (Genres: Comedy)


In [99]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Happy', 200)
for movie in recommended_movies:
    print(movie)


My Heavenly City (Genres: Romance, Family)
Marutha (Genres: Family, Romance)
All These Years (Genres: Family, Romance)
Anni Manchi Sakunamule (Genres: Comedy, Family, Romance)
Phalguna Chaitra (Genres: Family, Romance, Drama)
Sridevi Shoban Babu (Genres: Romance, Comedy, Family)
Haar Mana Haar (Genres: Drama, Family, Romance)
Holly and the Hot Chocolate (Genres: Family, Romance, Comedy)
Erin's Guide to Kissing Girls (Genres: Comedy, Family, Romance)
Margazhi Thingal (Genres: Drama, Family, Romance)
Pattathu Arasan (Genres: Family, Action, Romance)
All I Want For Christmas (Genres: Comedy, Romance, Family)
Rudy (Genres: Drama, Family, Romance)
Ardhangini (Genres: Romance, Family, Drama)
Senior Year: Love Never Fails (Genres: Drama, Family, Romance)
Reunirse (Genres: Drama, Family, Romance)
Table for Six (Genres: Comedy, Family, Romance)
The Stars Will Guide Me (Genres: Music, Family, Romance)
Good Goodbye 好好说再见 (Genres: Family, Drama, Romance)
Kalyanam Kamaneeyam (Genres: Romance, Comed

In [100]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

Kuttram Purinthal (Genres: Crime)
Betrayal (Genres: Crime)
Zu jung zu sterben (Genres: Crime)
La Familia (Genres: Crime)
The Policeman's Lineage (Genres: Crime)
Saint (Genres: Crime)
Freaky Profiling (Genres: Crime)
Alone Today (Genres: Crime)
Inspector Dupin: Brittany's Nights (Genres: Crime)
Target Number One (Genres: Crime)


In [101]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

Armageddon Code (Genres: Action, Adventure)
Borders (Genres: Action, Adventure)
Sayen (Genres: Action, Adventure)
The Siege of Robin Hood (Genres: Adventure, Action)
Home-Sitters (Genres: Adventure, Action)
Viman (Genres: Adventure, Action)
Tozoom (Genres: Adventure, Action)
Sniper: Vengeance (Genres: Action, Adventure)
Raiders of the Lost Library (Genres: Action, Adventure)
The Fourth Musketeer (Genres: Action, Adventure)


In [102]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Happy', 10)
for movie in recommended_movies:
    print(movie)

Human Nature (Genres: Adventure)
Baby Boy of House (Genres: Adventure)
Always Now (Genres: Adventure)
Vestige (Genres: Adventure)
Dreaming of mother nature (Genres: Adventure)
La Ciudad de los Cesares (Genres: Adventure)
Escape Through Africa (Genres: Adventure)
We Hunt Giants (Genres: Adventure)
Catastrophe, Inc. (Genres: Adventure)
Fisch im Fell (Genres: Adventure)


In [103]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

Amityville in Space (Genres: Horror, Science Fiction)
M3GAN (Genres: Science Fiction, Horror)
Big Brood (Genres: Horror, Science Fiction)
Curse Of The Re-Animator (Genres: Horror, Science Fiction)
The October Martyr (Genres: Science Fiction, Horror)
CD-Trip (Genres: Horror, Science Fiction)
Cocaine Shark (Genres: Horror, Science Fiction)
The Ghost Lights (Genres: Horror, Science Fiction)
Iron Lung (Genres: Horror, Science Fiction)
Critters: All You Can Eat (Genres: Horror, Science Fiction)


In [104]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

Kuttram Purinthal (Genres: Crime)
Betrayal (Genres: Crime)
Zu jung zu sterben (Genres: Crime)
La Familia (Genres: Crime)
The Policeman's Lineage (Genres: Crime)
Saint (Genres: Crime)
Freaky Profiling (Genres: Crime)
Alone Today (Genres: Crime)
Inspector Dupin: Brittany's Nights (Genres: Crime)
Target Number One (Genres: Crime)


In [105]:
user

,id,happy_movie,sad_movie,neutral_movie,fav_movie
0,1,"[Drama, Comedy]","[Drama, Thriller]",[Comedy],The Godfather
1,2,"[Romance, Family]",[Crime],"[Action, Adventure]",Schindler's List
2,3,[Adventure],"[Horror, Science Fiction]",[Crime],The Shawshank Redemption


Relevant code ends here